##  running MPI4py on EC2 using spotpricing
#### For general tutorial of running running MPI4py this tutorial is more linked towards spotpricing


In [1]:
%%bash
. ~/.bashrc
pip install  --upgrade git+https://git@github.com/JonasWallin/linkingEC2



  Cloning https://git@github.com/JonasWallin/linkingEC2 to /var/folders/ct/zpjk2t696k5bv03sqq9xrpd00000gn/T/pip-Bj9Acz-build
Requirement already up-to-date: boto in /Users/jonaswallin/anaconda/lib/python2.7/site-packages (from linkingEC2==0.1)
  Found existing installation: linkingEC2 0.1
    Uninstalling linkingEC2-0.1:
      Successfully uninstalled linkingEC2-0.1
  Running setup.py install for linkingEC2


In [2]:
## STARTING UPP LINKER

from ConfigParser import ConfigParser
config = ConfigParser()
starfigconfig_folder = "/Users/jonaswallin/.starcluster/"
config.read(starfigconfig_folder + "config")

acess_key_id     = config.get('aws info', 'aws_access_key_id'    , 0)
aws_secret_key   = config.get('aws info', 'aws_secret_access_key', 0)
aws_region_name  = config.get('aws info', 'aws_region_name'      , 0)
my_key_loc       = config.get('key mykeyABC', 'key_location',0)
from linkingEC2 import LinkingHandler
linker = LinkingHandler(aws_secret_access_key = aws_secret_key,
                        aws_access_key_id     = acess_key_id,
                        aws_region_name       = aws_region_name,
                        key_location          = my_key_loc,
                        key_name              = 'mykeyABC')

### Here one need to have created some spotprices!

In [3]:
linker.connect_spot_instances()

AttributeError: 'NoneType' object has no attribute 'append'

In [ ]:
import time
PACKAGES_APT = ['python-dev', 'openmpi-bin', 'libopenmpi-dev', 'python-numpy']
PACKAGES_PIP = ['mpi4py']
time_in = time.time()
linker.apt_install(PACKAGES_APT)
linker.pip_install(PACKAGES_PIP)
time_out = time.time()
import os
os.system('say "your packages is downloaded"')
print('it took {time:.2f} sec'.format(time = time_out - time_in))

In [4]:
%%writefile calc_pi_MPI.py
from mpi4py import MPI
import numpy as np
import random
comm = MPI.COMM_WORLD
rank = comm.Get_rank()
mpisize = comm.Get_size()
nsamples = int(12e6/mpisize)
inside = 0
random.seed(rank)
for i in range(nsamples):
    x = random.random()
    y = random.random()
if (x*x)+(y*y)<1:
    inside += 1
mypi = (4.0 * inside)/nsamples
pi = comm.reduce(mypi, op=MPI.SUM, root=0)
if rank==0:
    print (1.0 / mpisize)*pi

Overwriting calc_pi_MPI.py


In [5]:
linker.conn.get_all_spot_instance_requests()

[SpotInstanceRequest:sir-021kesj1,
 SpotInstanceRequest:sir-021l0tb4,
 SpotInstanceRequest:sir-021j83dz,
 SpotInstanceRequest:sir-021rpk5s]